In [1]:
import fsspec
import xarray as xr

import pandas as pd

In [2]:
filepath = 'https://power-analysis-ready-datastore.s3.amazonaws.com/power_901_monthly_meteorology_utc.zarr'
filepath_mapped = fsspec.get_mapper(filepath)

ds = xr.open_zarr(store=filepath_mapped, consolidated=True)
ds

<xarray.Dataset>
Dimensions:          (time: 492, lat: 361, lon: 576)
Coordinates:
  * lat              (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * lon              (lon) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * time             (time) datetime64[ns] 1981-01-31 1981-02-28 ... 2021-12-31
Data variables: (12/94)
    CDD0             (time, lat, lon) float64 ...
    CDD10            (time, lat, lon) float64 ...
    CDD18_3          (time, lat, lon) float64 ...
    DISPH            (time, lat, lon) float64 ...
    EVLAND           (time, lat, lon) float64 ...
    EVPTRNS          (time, lat, lon) float64 ...
    ...               ...
    WS50M_MAX_AVG    (time, lat, lon) float64 ...
    WS50M_MIN        (time, lat, lon) float64 ...
    WS50M_MIN_AVG    (time, lat, lon) float64 ...
    WS50M_RANGE      (time, lat, lon) float64 ...
    WS50M_RANGE_AVG  (time, lat, lon) float64 ...
    Z0M              (time, lat, lon) float64 ...
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2022-04-11T12:20:00.000000
    ...                         ...
    time_coverage_end:          2021-12-31T00:00:00
    time_coverage_resolution:   P1M
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Monthly Meteorology Time Series
    version:                    POWER Version 9.0.1

In [3]:
# SOURCE = /power_901_monthly_meteorology_utc.zarr/
# TSURF -------
# long_name     : Surface Temperature of Land and Snow
# standard_name : Surface_Temperature_of_Land_and_Snow
# units         : K
# valid_max     : 350.0
# valid_min     : 150.0
# valid_range   : 150.0, 350.0]

# GWETTOP -----
# long_name     : Surface Soil Wetness
# standard_name : Surface_Soil_Wetness
# units         : 1
# valid_max     : 1.0
# valid_min     : 0.0
# valid_range   : [0.0, 1.0]

# PRECSNO --------
# long_name     : Snow Precipitation
# standard_name : Snow_Precipitation
# units         : kg m-2 s-1
# valid_max     : 0.0005
# valid_min     : 0.0
# valid_range   : [0.0, 0.0005]

# RH2M -------
# long_name     : Relative Humidity at 2 Meters
# standard_name : Relative_Humidity_at_2_Meters
# units         : %
# valid_max     : 100.0
# valid_min     : 0.0
# valid_range   : [0.0, 100.0]

# WS2M -------
# long_name     : Wind Speed at 2 Meters
# standard_name : Wind_Speed_at_2_Meters
# units         : m/s
# valid_max     : 50.0
# valid_min     : 0.0
# valid_range   : [0.0, 50.0]

# Source = /power_901_daily_precipitation_utc.zarr/
# PRECIPITATIONCAL ------
# long_name : The accumulated precipitation from all available infrared (IR) and microwave (MW) sources.
# units     : mm/day

## Select Columns of Interest

In [4]:
selected = ds[["TSURF", "GWETTOP", "PRECSNO", "RH2M", "WS2M"]]
selected

<xarray.Dataset>
Dimensions:  (time: 492, lat: 361, lon: 576)
Coordinates:
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * time     (time) datetime64[ns] 1981-01-31 1981-02-28 ... 2021-12-31
Data variables:
    TSURF    (time, lat, lon) float64 ...
    GWETTOP  (time, lat, lon) float64 ...
    PRECSNO  (time, lat, lon) float64 ...
    RH2M     (time, lat, lon) float64 ...
    WS2M     (time, lat, lon) float64 ...
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2022-04-11T12:20:00.000000
    ...                         ...
    time_coverage_end:          2021-12-31T00:00:00
    time_coverage_resolution:   P1M
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Monthly Meteorology Time Series
    version:                    POWER Version 9.0.1

## Select Data from 2015-2021
We also drop NA values which are readings of the sea.

In [5]:
all_data = selected.sel(time=slice('2021-01-01', '2021-12-31')).to_dataframe().reset_index().dropna()
all_data = all_data[all_data["time"] > "2015"]
all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
34806,2021-01-31,-60.0,-26.250,274.031250,1.000000,0.000031,92.4375,6.789062
35378,2021-01-31,-59.5,-28.750,274.078125,1.000000,0.000030,92.5000,6.929688
35379,2021-01-31,-59.5,-28.125,274.078125,0.968750,0.000030,92.5625,6.921875
35380,2021-01-31,-59.5,-27.500,274.070312,0.929688,0.000030,92.5625,6.929688
35381,2021-01-31,-59.5,-26.875,274.039062,0.921875,0.000030,92.5000,6.953125
...,...,...,...,...,...,...,...,...
2487972,2021-12-31,84.0,-37.500,244.812500,0.984375,0.000005,97.1250,5.078125
2487973,2021-12-31,84.0,-36.875,244.812500,0.984375,0.000005,97.3750,5.117188
2487974,2021-12-31,84.0,-36.250,244.812500,0.984375,0.000006,97.6250,5.156250
2487975,2021-12-31,84.0,-35.625,244.812500,0.992188,0.000006,97.7500,5.195312


In [10]:
all_data.to_csv("./all_data_2021.csv")

In [11]:
all_data[all_data["time"] == "2021-01-31"]

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
34806,2021-01-31,-60.0,-26.250,274.031250,1.000000,0.000031,92.4375,6.789062
35378,2021-01-31,-59.5,-28.750,274.078125,1.000000,0.000030,92.5000,6.929688
35379,2021-01-31,-59.5,-28.125,274.078125,0.968750,0.000030,92.5625,6.921875
35380,2021-01-31,-59.5,-27.500,274.070312,0.929688,0.000030,92.5625,6.929688
35381,2021-01-31,-59.5,-26.875,274.039062,0.921875,0.000030,92.5000,6.953125
...,...,...,...,...,...,...,...,...
200676,2021-01-31,84.0,-37.500,243.265625,0.984375,0.000003,93.5000,4.203125
200677,2021-01-31,84.0,-36.875,243.265625,0.984375,0.000003,93.8750,4.273438
200678,2021-01-31,84.0,-36.250,243.265625,0.984375,0.000003,94.0625,4.343750
200679,2021-01-31,84.0,-35.625,243.265625,0.992188,0.000004,94.2500,4.414062


In [12]:
(all_data[all_data["time"] == "2021-01-31"]).to_csv("./all_data_jan_2021.csv")